# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

### Firstly, import the libraries

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [6]:
from bs4 import BeautifulSoup
import urllib.request

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = urllib.request.urlopen(url)
article = req.read().decode()

with open('ISO_3166-1_alpha-2.html', 'w') as fo:
    fo.write(article)
    
soup = BeautifulSoup(article, 'html.parser')

### obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe df

In [7]:
tables = soup.find_all('table', class_='sortable')
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:3] == ['Postcode', 'Borough', 'Neighbourhood']:
        break

In [8]:
df = pd.DataFrame(columns=headings)
i=0

for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        postcode, borough, neighbourhood = [td.text.strip() for td in tds[:3]]
        
        # Ignore cells with a borough that is Not assigned.
        if 'Not assigned' in borough:
            continue
        # If a cell has a borough but a Not assigned neighborhood, 
        # then the neighborhood will be the same as the borough. 
        if 'Not assigned' in neighbourhood:
            neighbourhood = borough
        df.loc[i] = [postcode, borough, neighbourhood]
        i=i+1

In [9]:
from functools import reduce

def to_set(x):
    setx = set(x)
    strs = ', '.join(setx)   
    return strs

#grouped = df.groupby("Postcode").agg({'Neighbourhood': to_set})

grouped = df.groupby("Postcode").agg({'Borough':'first', 'Neighbourhood': to_set})

df2 = df.drop(columns='Neighbourhood')
grouped = grouped.reset_index()

#grouped = pd.merge(df2, grouped, how='inner', on='Postcode')

    
#grouped = pd.DataFrame({'Qty_cnt' : df['Borough'], grouped]

grouped[['Postcode','Borough','Neighbourhood']]

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Highland Creek, Port Union"
2,M1E,Scarborough,"Morningside, West Hill, Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Ionview, Kennedy Park, East Birchmount Park"
7,M1L,Scarborough,"Oakridge, Clairlea, Golden Mile"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [10]:
grouped.shape

(103, 3)

### Part 2:  get the latitude and the longitude coordinates of each neighborhood.

In [20]:
!conda install -c conda-forge geocoder  #install geocoder for the 1st run
import geocoder # import geocoder

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geocoder:   1.38.1-py_0  conda-forge
    orderedset: 2.0-py35_0   conda-forge
    ratelim:    0.1.6-py35_0 conda-forge

orderedset-2.0 100% |################################| Time: 0:00:00  57.66 MB/s
ratelim-0.1.6- 100% |################################| Time: 0:00:00  12.92 MB/s
geocoder-1.38. 100% |################################| Time: 0:00:00  47.98 MB/s


In [29]:
# Comment:
# I tried to use geocoder, but the API seems not working, 
# so I choose to use the csv file instead
#
#nbh = 'Malvern, Rouge'
#postal_code = 'M1B'
#g = geocoder.google(nbh)
#lat_lng_coords = g.latlng

### Import lat_lng_coords CSV

In [44]:
df_ll = pd.read_csv("https://cocl.us/Geospatial_data")

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]
g2 = grouped[['Postcode','Borough','Neighbourhood']]
d2 = df_ll.rename(index=str, columns={"Postal Code": "Postcode"})
g2 = pd.merge(d2, g2, how='inner', on='Postcode')

g2 = g2[['Postcode','Borough','Neighbourhood', 'Latitude', 'Longitude']]
g2


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Highland Creek, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, West Hill, Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview, Kennedy Park, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Oakridge, Clairlea, Golden Mile",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
